# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv", dtype="object", encoding="utf-8")
df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,saint-philippe,RE,-21.3585,55.7679,71.19,75,48,16.64,1627743291
1,chimbote,PE,-9.0853,-78.5783,67.33,77,40,8.93,1627743291
2,porosozero,RU,62.7191,32.727,65.17,77,23,5.77,1627743292
3,manyana,BW,-23.4,21.7167,69.67,18,0,13.09,1627743292
4,guerrero negro,MX,27.9769,-114.0611,73.42,72,53,4.7,1627743292


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)
humidity = df['Humidity'].astype(float)

In [4]:
#plot heatmap
fig = gmaps.figure(center = (15,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 85 degrees but higher than 70.
    * Wind speed less than 20 mph.
    * Cloudiness less than 10%.
* Drop any rows will null values.

In [12]:
temp=df['Max Temp'].astype(float)
wind=df['Wind Speed'].astype(float)
cloud=df['Cloudiness'].astype(float)

perfect=df.loc[(temp > 70) & (temp < 85)] 
perfect=perfect.loc[(wind < 10)]
perfect=perfect.loc[(cloud < 1)]
perfect.dropna()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
11,cape town,ZA,-33.9258,18.4232,71.11,58,0,8.05,1627743118
22,porto,PT,41.1496,-8.611,74.28,60,0,4.0,1627743297
47,east london,ZA,-33.0153,27.9116,77.94,33,0,5.75,1627743306
109,saskylakh,RU,71.9167,114.0833,70.27,40,0,8.81,1627743325
116,port elizabeth,ZA,-33.918,25.5701,77.29,36,0,5.75,1627743327
159,ponta do sol,PT,32.6667,-17.1,82.71,69,0,5.48,1627743341
240,anqing,CN,30.5092,117.0506,82.56,79,0,2.93,1627743364
262,guangyuan,CN,32.442,105.823,79.79,88,0,3.38,1627743370
334,balkhash,KZ,46.844,74.9804,75.43,39,0,2.59,1627743392
362,bay village,US,41.4848,-81.9221,73.11,61,0,2.37,1627743400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df=pd.DataFrame(perfect).reset_index()
hotel_df['Hotel Name']=" "
hotel_df

,index,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,11,cape town,ZA,-33.9258,18.4232,71.11,58,0,8.05,1627743118,
1,22,porto,PT,41.1496,-8.611,74.28,60,0,4.0,1627743297,
2,47,east london,ZA,-33.0153,27.9116,77.94,33,0,5.75,1627743306,
3,109,saskylakh,RU,71.9167,114.0833,70.27,40,0,8.81,1627743325,
4,116,port elizabeth,ZA,-33.918,25.5701,77.29,36,0,5.75,1627743327,
5,159,ponta do sol,PT,32.6667,-17.1,82.71,69,0,5.48,1627743341,
6,240,anqing,CN,30.5092,117.0506,82.56,79,0,2.93,1627743364,
7,262,guangyuan,CN,32.442,105.823,79.79,88,0,3.38,1627743370,
8,267,oranjemund,NaN,-28.55,16.4333,81.55,18,0,9.33,1627743372,
9,334,balkhash,KZ,46.844,74.9804,75.43,39,0,2.59,1627743392,


In [43]:
url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
params = {"radius": 5000,
         "keyword": "hotel", 
         "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    city_name=row['City']
    
    params["location"] = f"{lat},{lng}"
    
    address= requests.get(url, params=params).json()
    results = address['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Could Not Find")
    print(results[0]['name'])

One&Only Cape Town
The Yeatman
Southern Sun Hemingways
Sberbank
Island Way Villa
Palace of Zinos
Anqing Yin Bin Hotel
Wanda Realm Guangyuan
Oryx Guesthouse Oranjemund
Гостиница Respect
DoubleTree by Hilton Hotel Cleveland - Westlake
Oui Hotel
Sevgi Otel
Hotel Durval


In [44]:
#drop locations without hotel name
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ' ']
hotel_df

,index,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,11,cape town,ZA,-33.9258,18.4232,71.11,58,0,8.05,1627743118,One&Only Cape Town
1,22,porto,PT,41.1496,-8.611,74.28,60,0,4.0,1627743297,The Yeatman
2,47,east london,ZA,-33.0153,27.9116,77.94,33,0,5.75,1627743306,Southern Sun Hemingways
3,109,saskylakh,RU,71.9167,114.0833,70.27,40,0,8.81,1627743325,Sberbank
4,116,port elizabeth,ZA,-33.918,25.5701,77.29,36,0,5.75,1627743327,Island Way Villa
5,159,ponta do sol,PT,32.6667,-17.1,82.71,69,0,5.48,1627743341,Palace of Zinos
6,240,anqing,CN,30.5092,117.0506,82.56,79,0,2.93,1627743364,Anqing Yin Bin Hotel
7,262,guangyuan,CN,32.442,105.823,79.79,88,0,3.38,1627743370,Wanda Realm Guangyuan
8,267,oranjemund,NaN,-28.55,16.4333,81.55,18,0,9.33,1627743372,Oryx Guesthouse Oranjemund
9,334,balkhash,KZ,46.844,74.9804,75.43,39,0,2.59,1627743392,Гостиница Respect


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'